<a href="https://colab.research.google.com/github/Ataerman/LLM_RAG-AI_AGENT/blob/main/Agent_with_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install langchain langchain-openai langchain-cohere langchain-community \
sentence-transformers faiss-cpu \
datasets accelerate \
tiktoken \
cohere


In [31]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
openai_api_key=userdata.get('openai')
cohere_api_key=userdata.get('cohere')
openai_llm=ChatOpenAI(model='gpt-3.5-turbo', #creating llm model (using chatgpt 3.5 with api)
                      openai_api_key=openai_api_key,
                      temperature=0.3)

In [32]:
text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following, and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

doc = Document(page_content=text)
# 🔹 Text Splitter: Break the text into smaller chunks with overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = splitter.split_documents([doc])
print(docs[0].page_content)

# 🔹 Embedding Model: Convert chunks into numerical vector representations
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 🔹 FAISS Vectorstore: Store and index the embeddings for similarity search
vectorstore = FAISS.from_documents(docs, embeddings)

Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher


In [34]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain.chains import RetrievalQA

# 🔹 Retriever: First retrieve top 10 similar chunks with FAISS
# 🔹 Then rerank them using Cohere Rerank API to improve relevance
retriever = ContextualCompressionRetriever(
    base_compressor=CohereRerank(
        cohere_api_key=cohere_api_key,
        model='rerank-multilingual-v2.0',
        top_n=3  # Return top 3 reranked chunks
    ),
    base_retriever=vectorstore.as_retriever(search_kwargs={'k': 10})
)
qa_chain=RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    chain_type="stuff")

In [35]:
!pip install duckduckgo-search

In [36]:
from langchain.agents import create_react_agent,AgentExecutor,load_tools
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import Tool
from langchain.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain.prompts import ChatPromptTemplate
rag_tool=Tool(
    name='Rag-Tool',
    description='Useful for answering questions about Interstellar',
    func=qa_chain.invoke)
search=DuckDuckGoSearchResults()
search_tool=Tool(
    name='duckduck',
    description='A web search engine. Use this to answer general knowledge and real-time queries.',
    func=search.run)
tools=load_tools(['llm-math'],llm=openai_llm)
tools+=[rag_tool,search_tool]

memory=ConversationBufferWindowMemory(
    memory_key='chat_history',
    return_messages=True,
    k=10)

In [37]:
from langchain import PromptTemplate
#Create ReAct Template
react_template = """
Answer the following questions as best you can. You have access to the following tools:
{tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
...(this Thought/Action/Action Input/Observation can repeat N times)
Thought: I know the final answer
Final Answer: the final answer to the original input question

Previous conversation:
{chat_history}

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt=PromptTemplate(template=react_template,
                      input_variables=['tools','tool_names','input','agent_scratchpad'])


agent=create_react_agent(openai_llm,tools,prompt)
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)


In [38]:
response = agent_executor.invoke({"input": "Who was the scientific consultant for the movie Interstellar?"})
print(response["output"])




> Entering new AgentExecutor chain...
Thought: This question is related to the movie Interstellar, so I should use the Rag-Tool to find the answer.
Action: Rag-Tool
Action Input: {"input": "Who was the scientific consultant for the movie Interstellar?"}{'query': '{"input": "Who was the scientific consultant for the movie Interstellar?"}', 'result': 'The scientific consultant for the movie Interstellar was Kip Thorne.'}Final Answer: The scientific consultant for the movie Interstellar was Kip Thorne.

> Finished chain.
The scientific consultant for the movie Interstellar was Kip Thorne.


In [39]:
response = agent_executor.invoke({"input": "What is the square root of 144 ?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: This is a math question that requires calculating the square root.
Action: Calculator
Action Input: 144Answer: 144Final Answer: The square root of 144 is 12.

> Finished chain.
The square root of 144 is 12.


In [40]:
response = agent_executor.invoke({"input": "Who won the Best Actor Oscar in 2024?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: This is a question about general knowledge.
Action: duckduck
Action Input: "Best Actor Oscar winner 2024"snippet: At the 2024 Oscars, Cillian Murphy won the best actor Oscar for his role of physicist J. Robert Oppenheimer, known as "the father of the atomic bomb." Watch Party Newsletter Best TV shows of 2025 ..., title: Who won the best actor Oscar last year? See list of recent winners, link: https://www.usatoday.com/story/entertainment/movies/2025/02/19/oscar-best-actor-2024/78472302007/, snippet: The Best Actor award has been presented 97 times, to 86 actors. The first winner was German actor Emil Jannings for his roles in The Last Command (1928) and The Way of All Flesh (1927). [ 1 ] The most recent winner is Adrien Brody for The Brutalist (2024); he previously won the award for The Pianist (2002) at the age of 29, making him the ..., title: Academy Award for Best Actor - Wikipedia, link: https://en.wikipedia.org/wiki/Academy_Award_fo

In [41]:
response = agent_executor.invoke({"input": "What was the my question about math ?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To determine the question about math, we need to recall the previous conversation.
Action: Rag-Tool
Action Input: {"input": "What is the square root of 144 ?"}{'query': '{"input": "What is the square root of 144 ?"}', 'result': 'The square root of 144 is 12.'}I know the final answer
Final Answer: The question about math was "What is the square root of 144?"

> Finished chain.
The question about math was "What is the square root of 144?"
